In [119]:
import pandas as pd
import pyterrier as pt
import os
import requests
import json
import re
import csv
import numpy as np

In [120]:
if not pt.started():
  pt.init()

In [121]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './indices/cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
  indexer = pt.index.IterDictIndexer(pt_index_path, blocks = True)
  index_ref = indexer.index(dataset.get_corpus_iter(),
                            fields = ['title', 'doi', 'abstract'],
                            meta = ('docno',))
else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

In [122]:
metadata = pd.read_csv('~/.ir_datasets/cord19/2020-07-16/metadata.csv')
#splitted_authors = pd.read_csv('./split_authors.csv')

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [123]:
len(metadata)

192509

In [124]:
metadata.head(5)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


Dropping empty autors

In [125]:
metadata = metadata[metadata['authors'].notna()]
metadata = metadata.reset_index()
metadata = metadata.drop(columns = 'index')

In [126]:
journal_abbrev = pd.read_csv('./data/wos_abbrev_table.csv', sep = ';', header = None)
journal_abbrev=journal_abbrev.T.set_index(0).T
journal_abbrev = journal_abbrev[['full', 'abbrev']]

In [127]:
metadata = pd.merge(metadata, journal_abbrev, left_on='journal', right_on='abbrev', how='left')
metadata['full'] = metadata['full'].fillna(metadata['journal'])


In [128]:
metadata

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,full,abbrev
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,BMC Infect Dis,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,Respiratory Research,Respir Res
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,Respiratory Research,Respir Res
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,Respiratory Research,Respir Res
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,Respiratory Research,Respir Res
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194688,z4ro6lmh,203f36475be74229101548475d68352b939f8b5b,Medline; PMC,Rapid radiological improvement of COVID-19 pne...,10.1007/s15010-020-01449-w,PMC7299451,32557206,no-cc,NaN,2020-06-15,...,Infection,NaN,NaN,NaN,document_parses/pdf_json/203f36475be7422910154...,document_parses/pmc_json/PMC7299451.xml.json,https://doi.org/10.1007/s15010-020-01449-w; ht...,219729576.0,Infection,Infection
194689,hi8k8wvb,9f1bc99798e8823e690697394dcb23533a45c60e,Elsevier; Medline; PMC,SARS E protein in phospholipid bilayers: an an...,10.1016/j.physb.2004.11.015,PMC7127356,32288217,els-covid,Abstract We report on an anomalous X-ray refle...,2005-02-28,...,Physica B: Condensed Matter,NaN,NaN,NaN,document_parses/pdf_json/9f1bc99798e8823e69069...,document_parses/pmc_json/PMC7127356.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32288217/;...,122247693.0,Physica B: Condensed Matter,NaN
194690,ma3ndg41,ffba777376718ef2a0dd74a8eab90e2bfacd240f,Medline; PMC,Italian Society of Interventional Cardiology (...,10.1002/ccd.28888,PMC7228289,32223063,no-cc,COVID‐19 pandemic raised the issue to guarante...,2020-04-11,...,Catheter Cardiovasc Interv,NaN,NaN,NaN,document_parses/pdf_json/ffba777376718ef2a0dd7...,document_parses/pmc_json/PMC7228289.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32223063/;...,214715941.0,Catheter Cardiovasc Interv,NaN
194691,wh10285j,d521c5a2dcbd79a5be606fcf586b1e0448344172,Medline; PMC,"Nimble, Together: A Training Program's Respons...",10.1097/sla.0000000000003994,PMC7224622,32355117,cc-by-nc-nd,NaN,2020-04-29,...,Ann Surg,NaN,NaN,NaN,document_parses/pdf_json/d521c5a2dcbd79a5be606...,document_parses/pmc_json/PMC7224622.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32355117

## Splitting the authors into single columns

A function to split the author dict up into single columns. To make the retrieval process possible, commas, double spaces and other characters had to be removed without using a complex regex solution (in case a person hat a accent in their name for example.

In [129]:
def split_authors(n):
    # get first n articles
    author_uid_df = metadata[['authors', 'cord_uid']][:n]
    author_uid_df['authors'] = author_uid_df['authors'].apply(pd.Series)
    # stringsplit authors
    splitted_authors_df = author_uid_df['authors'].str.split(';', expand=True)
    author_uid_df = pd.concat([author_uid_df, splitted_authors_df], axis=1)
    # drop the authors (column with multiple authors)
    author_uid_df.drop(columns='authors')
    # replace the empty column of multiply authors
    #author_title_df = author_title_df.replace('', np.nan).set_index('title')
    author_uid_df = author_uid_df.drop(columns='authors')
    author_uid_df = author_uid_df.replace('', np.nan).set_index('cord_uid').stack().reset_index(name='author').drop('level_1',1)
    author_uid_df['author'] = author_uid_df['author'].str.split(',').str[::-1].str.join(' ')

    return author_uid_df

In [130]:
def split_authors(n):
    # get first n articles
    author_uid_df = metadata[['authors', 'cord_uid']][:n]
    author_uid_df['authors'] = author_uid_df['authors'].apply(pd.Series)
    # stringsplit authors
    splitted_authors_df = author_uid_df['authors'].str.split(';', expand=True)
    #splitted_authors_df = splitted_authors_df.replace(',','', regex=True)
    # concatenate the splitted authors to our dataframe
    author_uid_df = pd.concat([author_uid_df, splitted_authors_df], axis=1)
    # drop the authors (column with multiple authors)
    author_uid_df.drop(columns='authors')
    # replace the empty column of multiply authors
    #author_title_df = author_title_df.replace('', np.nan).set_index('title')
    author_uid_df = author_uid_df.drop(columns='authors')
    #author_title_df = author_title_df.reset_index()
    author_uid_df = author_uid_df.replace('', np.nan).set_index('cord_uid').stack().reset_index(name='author').drop('level_1',1)
    author_uid_df['author'] = author_uid_df['author'].str.split(',').str[::-1].str.join(' ')
    author_uid_df['author'] = author_uid_df['author'].str.replace('  ', ' ')
    author_uid_df['author'] = author_uid_df['author'].str.lstrip()
    merged_data = metadata[['cord_uid', 'title', 'license', 'publish_time', 'full']]
    merged_data = author_uid_df.merge(merged_data, on = 'cord_uid')
    return merged_data

### How to work with the API:

Basic request:
https://api.openalex.org/ + work/authors/venues/insitutions/concecpts + ?filter = + columns.search + Name

In [131]:
tariq = requests.get(
    'https://api.openalex.org/authors?filter=display_name.search:Tariq'
).json()['results'][0] 

#tariq                      

## Retrieving author information

A function to retrieve all the authors from the metadata's first n documents. It creates a list to call OpenAlex with.

In [132]:
def getAuthors(n_documents):
    author_list = []
    for i in range(n_documents):
        print(metadata['authors'][i])
        if(metadata['authors'][i] != "nan"):#isinstance(i,float)):
            authors_split = metadata['authors'][i].split(";")
            for j in range(len(authors_split)):
                #authors_split[j] = re.sub(r'\W+', ' ', authors_split[j])
                #authors_split[j] = authors_split[j].replace('-',' ')
                authors_split[j] = authors_split[j].replace(',', '')
                authors_split[j] = authors_split[j].lstrip()
                author_list.append(authors_split[j])
        else:
            author_list.append("")
    return author_list

In [133]:
authors = getAuthors(500)

Madani, Tariq A; Al-Ghamdi, Aisha A
Vliet, Albert van der; Eiserich, Jason P; Cross, Carroll E
Crouch, Erika C
Fagan, Karen A; McMurtry, Ivan F; Rodman, David M
Domachowske, Joseph B; Bonville, Cynthia A; Rosenberg, Helene F
Pasternak, Alexander O.; van den Born, Erwin; Spaan, Willy J.M.; Snijder, Eric J.
Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn
Ball, Jonathan; Venn, Richard
Slebos, Dirk-Jan; Ryter, Stefan W; Choi, Augustine MK
Tsui, Fu-Chiang; Espino, Jeremy U.; Dato, Virginia M.; Gesteland, Per H.; Hutman, Judith; Wagner, Michael M.
Ivanov, Ivaylo P.; Matsufuji, Senya; Murakami, Yasuko; Gesteland, Raymond F.; Atkins, John F.
Shi, Stephanie T.; Huang, Peiyong; Li, Hsin-Pai; Lai, Michael M.C.
Pridgeon, Julia W.; Geetha, Thangiah; Wooten, Marie W.
Ploubidou, Aspasia; Moreau, Violaine; Ashman, Keith; Reckmann, Inge; González, Cayetano; Way, Michael
Barry, John M
Shieh, Biehuoy; Li, Ching
Verheij, Joanne; Groeneveld, AB Johan; Beishuizen, Albertus; Lingen, Arthur van; Simoons-Smit,

In [134]:
authors

['Madani Tariq A',
 'Al-Ghamdi Aisha A',
 'Vliet Albert van der',
 'Eiserich Jason P',
 'Cross Carroll E',
 'Crouch Erika C',
 'Fagan Karen A',
 'McMurtry Ivan F',
 'Rodman David M',
 'Domachowske Joseph B',
 'Bonville Cynthia A',
 'Rosenberg Helene F',
 'Pasternak Alexander O.',
 'van den Born Erwin',
 'Spaan Willy J.M.',
 'Snijder Eric J.',
 'Alvarez Gonzalo',
 'Hébert Paul C',
 'Szick Sharyn',
 'Ball Jonathan',
 'Venn Richard',
 'Slebos Dirk-Jan',
 'Ryter Stefan W',
 'Choi Augustine MK',
 'Tsui Fu-Chiang',
 'Espino Jeremy U.',
 'Dato Virginia M.',
 'Gesteland Per H.',
 'Hutman Judith',
 'Wagner Michael M.',
 'Ivanov Ivaylo P.',
 'Matsufuji Senya',
 'Murakami Yasuko',
 'Gesteland Raymond F.',
 'Atkins John F.',
 'Shi Stephanie T.',
 'Huang Peiyong',
 'Li Hsin-Pai',
 'Lai Michael M.C.',
 'Pridgeon Julia W.',
 'Geetha Thangiah',
 'Wooten Marie W.',
 'Ploubidou Aspasia',
 'Moreau Violaine',
 'Ashman Keith',
 'Reckmann Inge',
 'González Cayetano',
 'Way Michael',
 'Barry John M',
 'Shieh

Retrieving the data using OpenAlex. If the author is not found, Empty Name is used as dummy.

In [135]:
len(authors)

2920

In [136]:
author_df = pd.DataFrame()

for i in range(len(authors)):
    try:
        author_information = requests.get(
            'https://api.openalex.org/authors?filter=display_name.search:'+authors[i]
        ).json()['results'][0]
        print('Retrieved:',authors[i])
        current_author = pd.DataFrame.from_dict(author_information, orient='index')
        current_author = current_author.transpose()
        author_df = author_df.append(current_author)
        author_df['display_name'] =  author_df['display_name'].str.lstrip()

        
    except:
        print("Empty name")
    #author_data = author_data.append(pd.json_normalize(author_information)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])
    #author_data[i] = author_information['x_concepts'][1]['display_name']
    
    #author_data.append(pd.json_normalize(tariq)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])


Retrieved: Madani Tariq A
Retrieved: Al-Ghamdi Aisha A
Retrieved: Vliet Albert van der
Retrieved: Eiserich Jason P
Retrieved: Cross Carroll E
Retrieved: Crouch Erika C
Retrieved: Fagan Karen A
Retrieved: McMurtry Ivan F
Retrieved: Rodman David M
Retrieved: Domachowske Joseph B
Retrieved: Bonville Cynthia A
Retrieved: Rosenberg Helene F
Retrieved: Pasternak Alexander O.
Retrieved: van den Born Erwin
Empty name
Retrieved: Snijder Eric J.
Retrieved: Alvarez Gonzalo
Retrieved: Hébert Paul C
Retrieved: Szick Sharyn
Retrieved: Ball Jonathan
Retrieved: Venn Richard
Retrieved: Slebos Dirk-Jan
Retrieved: Ryter Stefan W
Retrieved: Choi Augustine MK
Retrieved: Tsui Fu-Chiang
Retrieved: Espino Jeremy U.
Retrieved: Dato Virginia M.
Retrieved: Gesteland Per H.
Retrieved: Hutman Judith
Retrieved: Wagner Michael M.
Empty name
Retrieved: Matsufuji Senya
Retrieved: Murakami Yasuko
Retrieved: Gesteland Raymond F.
Retrieved: Atkins John F.
Retrieved: Shi Stephanie T.
Empty name
Retrieved: Li Hsin-Pai
Retr

In [137]:
author_df

,id,orcid,display_name,display_name_alternatives,relevance_score,works_count,cited_by_count,ids,last_known_institution,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/A2000431824,https://orcid.org/0000-0003-2453-0623,Tariq A. Madani,[Tariq A. Madani],3084.6128,107,5240,{'openalex': 'https://openalex.org/A2000431824...,"{'id': 'https://openalex.org/I185163786', 'ror...","[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-29T10:55:48.378800,2016-06-24
0,https://openalex.org/A2181556405,https://orcid.org/0000-0001-8106-0424,Aisha A. Al-Ghamdi,[],548.21704,29,122,{'openalex': 'https://openalex.org/A2181556405...,"{'id': 'https://openalex.org/I185163786', 'ror...","[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2022, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-26T16:04:40.263411,2016-06-24
0,https://openalex.org/A2134341530,https://orcid.org/0000-0003-0923-0016,Albert van der Vliet,[],5049.549,246,10445,{'openalex': 'https://openalex.org/A2134341530...,"{'id': 'https://openalex.org/I111236770', 'ror...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-27T13:35:17.849811,2016-06-24
0,https://openalex.org/A62037741,None,Jason P. Eiserich,[],4635.8936,133,8770,"{'openalex': 'https://openalex.org/A62037741',...","{'id': 'https://openalex.org/I84218800', 'ror'...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-28T18:55:11.482202,2016-06-24
0,https://openalex.org/A2098199652,None,Carroll E. Cross,[],4898.441,320,14245,{'openalex': 'https://openalex.org/A2098199652...,"{'id': 'https://openalex.org/I84218800', 'ror'...","[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-29T08:07:00.059938,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,https://openalex.org/A4222400290,None,Juliana C. N. Chan,[],76.98717,2,4,{'openalex': 'https://openalex.org/A4222400290'},"{'id': 'https://openalex.org/I177725633', 'ror...",[],"[{'year': 2023, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-28T01:25:56.400836,2022-04-06
0,https://openalex.org/A2573029335,https://orcid.org/0000-0002-3649-6680,Yuan Soon Ho,[],2978.1692,247,6202,{'openalex': 'https://openalex.org/A2573029335...,"{'id': 'https://openalex.org/I47519274', 'ror'...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-27T13:08:26.493590,2017-01-26
0,https://openalex.org/A3048868603,https://orcid.org/0000-0002-8859-5453,Lu Zhang,[],10081.909,10167,160998,{'openalex': 'https://openalex.org/A3048868603...,"{'id': 'https://openalex.org/I145897649', 'ror...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 8, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-30T07:52:16.015398,2020-08-18
0,https://openalex.org/A2198086772,https://orcid.org/0000-0002-2566-7851,Dongyue Li,[],1951.197,154,2655,{'openalex': 'https://openalex.org/A2198086772...,"{'id': 'https://openalex.org/I161318765', 'ror...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2023-01-27T07:51:48.194634,2016-06-24


Cleaning up the author data: 

1. Defining a dataframe
2. add the extracted information into our dataframe
3. replace special characters in the display name
4. add the x_concepts (most common research field of the author according to openalex)

In [138]:
author_data = pd.DataFrame(columns= [['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution', 'x_concepts']])
author_data = author_df[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution', 'x_concepts']]

author_data['research_field'] = ""
author_data = author_data.reset_index()
author_data = author_data.drop(columns = 'index')
#author_data['counter'] = range(len(author_data))
#author_data = author_data.drop(columns='index')

#clean display names again

for i in range(len(author_data)):
    #author_data['display_name'][i] = re.sub(r'\W+', ' ', author_data['display_name'][i])
    #authors_split[j] = authors_split[j].replace('-',' ')
    author_data['display_name'][i] = author_data['display_name'][i].lstrip()
#author_data.display_name = author_data.display_name.str.replace('[^a-zA-Z]', ' ')

for i in range(len(author_data)):
    try:
        author_data.iloc[i, author_data.columns.get_loc('research_field')] = author_data.iloc[i]['x_concepts'][0]['display_name']
    except:
        author_data.iloc[i, author_data.columns.get_loc('research_field')] = "Unknown"
    try:
        author_data.iloc[i, author_data.columns.get_loc('last_known_institution')] = (list(author_data['last_known_institution'])[i]['display_name'])    
    except:
        author_data.iloc[i, author_data.columns.get_loc('last_known_institution')] = "Unknown"
author_data = author_data.drop(columns = ['x_concepts']) 

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/3892613631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_data['research_field'] = ""


In [139]:
author_data

,display_name,relevance_score,works_count,cited_by_count,last_known_institution,research_field
0,Tariq A. Madani,3084.6128,107,5240,King Abdulaziz University,Medicine
1,Aisha A. Al-Ghamdi,548.21704,29,122,King Abdulaziz University,Medicine
2,Albert van der Vliet,5049.549,246,10445,University of Vermont,Biology
3,Jason P. Eiserich,4635.8936,133,8770,"University of California, Davis",Biology
4,Carroll E. Cross,4898.441,320,14245,"University of California, Davis",Medicine
...,...,...,...,...,...,...
2765,Juliana C. N. Chan,76.98717,2,4,Chinese University of Hong Kong,Unknown
2766,Yuan Soon Ho,2978.1692,247,6202,Taipei Medical University,Biology
2767,Lu Zhang,10081.909,10167,160998,Minzu University of China,Biology
2768,Dongyue Li,1951.197,154,2655,"University of California, Los Angeles",Physics


## Retrieving Insitution Information

Create the institution list from the author data

In [140]:
institutions_list = author_data['last_known_institution'].unique()

Replace commas and spaces at the start to retrieve the documents.

In [141]:
for i in range(len(institutions_list)):
    institutions_list[i] = institutions_list[i].replace(',','')
    institutions_list[i] = institutions_list[i].lstrip()
    print(institutions_list[i])
    #authors_split[j].replace(',','')
     #       authors_split[j] = authors_split[j].lstrip()
      #      author_list.append(authors_split[j])

King Abdulaziz University
University of Vermont
University of California Davis
Washington University in St. Louis
University of South Alabama
ProQR Therapeutics
SUNY Upstate Medical University
National Institute of Allergy and Infectious Diseases
University of Amsterdam
MSD
Leiden University Medical Center
Oak Ridge National Laboratory
University of Montreal
Ottawa Hospital
University of Nottingham
Worthing Hospital
University Medical Center Groningen
Cornell College
Unknown
Children's Hospital of Philadelphia
University of Pittsburgh
University of Utah
Jikei University School of Medicine
Howard Hughes Medical Institute
University College Cork
Pfizer
Chang Gung University
Agricultural Research Service
Auburn University
Leibniz Institute on Aging – Fritz Lipmann Institute
Inserm
University of Queensland
European Molecular Biology Laboratory
Institució Catalana de Recerca i Estudis Avançats
The Francis Crick Institute
Oregon Medical Research Center
University of Colorado Denver
Bloomberg

Create a dataframe and retrieve the institutions using OpenAlex.

In [142]:
institution_df = pd.DataFrame()

for i in range(len(institutions_list)):
    try:
        print('Retrieved:',institutions_list[i])
        institution_information = requests.get(
            'https://api.openalex.org/institutions?filter=display_name.search:'+institutions_list[i]
        ).json()['results'][0]
        current_institution = pd.DataFrame.from_dict(institution_information, orient='index')
        current_institution = current_institution.transpose()
        institution_df = institution_df.append(current_institution)
    except:
        print("Empty name")
    #author_data = author_data.append(pd.json_normalize(author_information)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])
    #author_data[i] = author_information['x_concepts'][1]['display_name']
    
    #author_data.append(pd.json_normalize(tariq)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])


Retrieved: King Abdulaziz University
Retrieved: University of Vermont
Retrieved: University of California Davis
Retrieved: Washington University in St. Louis
Retrieved: University of South Alabama
Retrieved: ProQR Therapeutics
Retrieved: SUNY Upstate Medical University
Retrieved: National Institute of Allergy and Infectious Diseases
Retrieved: University of Amsterdam
Retrieved: MSD
Retrieved: Leiden University Medical Center
Retrieved: Oak Ridge National Laboratory
Retrieved: University of Montreal
Retrieved: Ottawa Hospital
Retrieved: University of Nottingham
Retrieved: Worthing Hospital
Retrieved: University Medical Center Groningen
Retrieved: Cornell College
Retrieved: Unknown
Empty name
Retrieved: Children's Hospital of Philadelphia
Retrieved: University of Pittsburgh
Retrieved: University of Utah
Retrieved: Jikei University School of Medicine
Retrieved: Howard Hughes Medical Institute
Retrieved: University College Cork
Retrieved: Pfizer
Retrieved: Chang Gung University
Retrieved: 

Preprocess the institution data by:

1. renaming
2. find out the main research field
3. replacing special characters

In [143]:
institution_data = institution_df[['display_name', 'relevance_score', 'country_code', 'type', 'cited_by_count', 'x_concepts']]
institution_data = institution_data.rename(columns= {'display_name' : 'institution_name', 'relevance_score': 'institution_relevance_score', 'cited_by_count': 'institution_cited_by_count', 'x_concepts' : 'institution_main_research_field'})

# gain knowledge about the institutions main research field
for i in range(len(institution_data)):
    try:
        institution_data.iloc[i, institution_data.columns.get_loc('institution_main_research_field')] = (list(institution_data['institution_main_research_field'])[i][0]['display_name'])    
    except:
        institution_data.iloc[i, institution_data.columns.get_loc('institution_main_research_field')] = "Unknown"

institution_data = institution_data.reset_index()
institution_data = institution_data.drop(columns = 'index')
        
institution_data.institution_name = institution_data.institution_name.str.replace('[^a-zA-Z]', ' ')
institution_data.institution_name = institution_data.institution_name.str.replace('  ',' ')

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/1874173663.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  institution_data.institution_name = institution_data.institution_name.str.replace('[^a-zA-Z]', ' ')


In [144]:
institution_data

,institution_name,institution_relevance_score,country_code,type,institution_cited_by_count,institution_main_research_field
0,King Abdulaziz University,81210.69,SA,education,1896975,Biology
1,University of Vermont,64095.375,US,education,2829240,Biology
2,University of California Davis,214908.62,US,education,16045859,Biology
3,Washington University in St Louis,341988.2,US,education,25811646,Biology
4,University of South Alabama,43150.04,US,education,768239,Biology
...,...,...,...,...,...,...
1167,Changchun Institute of Applied Chemistry,79473.234,CN,facility,1547841,Chemistry
1168,Macau University of Science and Technology,29101.363,CN,education,275892,Computer science
1169,Jinzhong University,3698.9907,CN,education,6100,Physics
1170,Minzu University of China,22775.115,CN,education,170348,Biology


## Combining the dataframes:

Combining the data was a time consuming task due to smaller errors and mistakes.
1. taking the authors from the metadata and replace their special characters
2. taking the authors from the retrieved authors to gain more knowledge about them
3. compare lost data due to merges (e.g. missing values somewhere or wrong name alignments)
    - 3.1 since the metadata is taken document wise, high amounts of authors could be lost. Then, more have to be retrived
4. merging the data 
5. comparing lost data due to merges in terms of adding the institution
6. merging the data again now containing information about the Article, Authors and Institutions

In [145]:
#replace metadata authors 
metadata_authors = split_authors(500)#len(metadata))
#metadata_authors.author = metadata_authors.author.str.replace('[^a-zA-Z]', ' ')
metadata_authors.author = metadata_authors.author.str.replace('  ',' ')
metadata_authors['author'] = metadata_authors['author'].str.lstrip()
metadata_authors.author = metadata_authors.author.str.replace('.', '')

metadata_authors = metadata_authors.rename(columns={'author': 'display_name'})

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/1753438378.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  author_uid_df = author_uid_df.replace('', np.nan).set_index('cord_uid').stack().reset_index(name='author').drop('level_1',1)
/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/2701385723.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metadata_authors.author = metadata_authors.author.str.replace('.', '')


In [146]:
# renaming columns and replacing special characters e.g. accent
author_data.display_name = author_data.display_name.str.replace('.', '')
#author_data.last_known_institution = author_data.last_known_institution.str.replace('[^a-zA-Z]', ' ')
author_data.last_known_institution = author_data.last_known_institution.str.replace('  ',' ')

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/602828608.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  author_data.display_name = author_data.display_name.str.replace('.', '')


lost data due to name merges:

In [147]:
metadata_authors

,cord_uid,display_name,title,license,publish_time,full
0,ug7v899j,Tariq A Madani,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis
1,ug7v899j,Aisha A Al-Ghamdi,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis
2,02tnwd4m,Albert van der Vliet,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research
3,02tnwd4m,Jason P Eiserich,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research
4,02tnwd4m,Carroll E Cross,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research
...,...,...,...,...,...,...
2927,a18rch1f,Juliana C N Chan,Apolipoprotein M Gene (APOM) Polymorphism Modi...,cc-by,2011-02-24,PLoS One
2928,a18rch1f,Yuan-Yuan Ho,Apolipoprotein M Gene (APOM) Polymorphism Modi...,cc-by,2011-02-24,PLoS One
2929,0jyzk5kt,Lu Zhang,Non-Invasive Microstructure and Morphology Inv...,cc-by,2011-02-25,PLoS One
2930,0jyzk5kt,Dongyue Li,Non-Invasive Microstructure and Morphology Inv...,cc-by,2011-02-25,PLoS One


In [148]:
metadata_authors['display_name']

0             Tariq A Madani
1          Aisha A Al-Ghamdi
2       Albert van der Vliet
3           Jason P Eiserich
4            Carroll E Cross
                ...         
2927        Juliana C N Chan
2928            Yuan-Yuan Ho
2929                Lu Zhang
2930              Dongyue Li
2931             Shuqian Luo
Name: display_name, Length: 2932, dtype: object

In [149]:
author_data['display_name']

0             Tariq A Madani
1          Aisha A Al-Ghamdi
2       Albert van der Vliet
3           Jason P Eiserich
4            Carroll E Cross
                ...         
2765        Juliana C N Chan
2766            Yuan Soon Ho
2767                Lu Zhang
2768              Dongyue Li
2769             Shuqian Luo
Name: display_name, Length: 2770, dtype: object

Lost data due to authors: eventually high if split_authors has a higher value than getAuthors.

In [150]:
len(list(set(metadata_authors['display_name']) - set(author_data['display_name'])))

637

In [151]:
#merging our dataframes

combined_data = metadata_authors.merge(author_data, on = 'display_name', how = 'outer')
#combined_data2 = combined_data.merge(institution_data, left_on='last_known_institution', right_on = 'institution_name')

In [152]:
combined_data

,cord_uid,display_name,title,license,publish_time,full,relevance_score,works_count,cited_by_count,last_known_institution,research_field
0,ug7v899j,Tariq A Madani,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,3084.6128,107,5240,King Abdulaziz University,Medicine
1,ug7v899j,Aisha A Al-Ghamdi,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,548.21704,29,122,King Abdulaziz University,Medicine
2,02tnwd4m,Albert van der Vliet,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,5049.549,246,10445,University of Vermont,Biology
3,02tnwd4m,Jason P Eiserich,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,4635.8936,133,8770,"University of California, Davis",Biology
4,02tnwd4m,Carroll E Cross,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,4898.441,320,14245,"University of California, Davis",Medicine
...,...,...,...,...,...,...,...,...,...,...,...
3674,NaN,Wayne Jun-Wei Zhou,NaN,NaN,NaN,NaN,161.24713,6,27,Chinese University of Hong Kong,Genetics
3675,NaN,C W Ma Ronald,NaN,NaN,NaN,NaN,69.7828,6,6,Unknown,Genetics
3676,NaN,Tao Qian,NaN,NaN,NaN,NaN,5364.3755,664,6897,Macau University of Science and Technology,Physics
3677,NaN,Yu-Zhen Chen,NaN,NaN,NaN,NaN,2393.0342,218,4903,Jinzhong University,Biology


lost data due to institution:

In [153]:
list(set(combined_data['last_known_institution']) - set(institution_data['institution_name']))

[nan,
 'University of California, Los Angeles',
 'Baden-Wuerttemberg Cooperative State University',
 'Hospital Clínic de Barcelona',
 'Friedrich-Loeffler-Institut',
 'University of Illinois Urbana-Champaign',
 'Agence France-Presse',
 "Saint John's Health Center",
 'University of Paris',
 'Institució Catalana de Recerca i Estudis Avançats',
 'Instituto Nacional de Ciencias Médicas y Nutrición Salvador Zubirán',
 'National Cancer Centre',
 "Scotland's Rural College",
 'Universitat Politècnica de València',
 'Maria Skłodowska-Curie Memorial Cancer Center and Institute of Oncology',
 'Research Center Borstel - Leibniz Lung Center',
 'Délégation Paris 7',
 'Institute of Nuclear Medicine & Allied Sciences',
 'Aix-Marseille University',
 'Azienda Sanitaria Unità Locale di Reggio Emilia',
 'Technische Universität Braunschweig',
 'University of Duisburg-Essen',
 'Institute of Molecular Biology',
 'Chemisches und Veterinäruntersuchungsamt Karlsruhe',
 'University of Göttingen',
 'Méditerranée I

In [154]:
#institution_data.institution_name = institution_data.institution_name.str.replace('[^a-zA-Z]', ' ')
#institution_data.institution_name = institution_data.institution_name.str.replace('  ',' ')

In [155]:
combined_data2 = combined_data.merge(institution_data, left_on = 'last_known_institution', right_on = 'institution_name', how = 'outer')

In [156]:
combined_data2

,cord_uid,display_name,title,license,publish_time,full,relevance_score,works_count,cited_by_count,last_known_institution,research_field,institution_name,institution_relevance_score,country_code,type,institution_cited_by_count,institution_main_research_field
0,ug7v899j,Tariq A Madani,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,3084.6128,107,5240,King Abdulaziz University,Medicine,King Abdulaziz University,81210.69,SA,education,1896975,Biology
1,ug7v899j,Aisha A Al-Ghamdi,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,548.21704,29,122,King Abdulaziz University,Medicine,King Abdulaziz University,81210.69,SA,education,1896975,Biology
2,02tnwd4m,Albert van der Vliet,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,5049.549,246,10445,University of Vermont,Biology,University of Vermont,64095.375,US,education,2829240,Biology
3,ip32y0j5,Sean A Diehl,Generation of Human Antigen-Specific Monoclona...,cc-by,2010-10-04,PLoS One,2378.744,76,3518,University of Vermont,Medicine,University of Vermont,64095.375,US,education,2829240,Biology
4,NaN,Y Wai Lam,NaN,NaN,NaN,NaN,250.73355,1,36,University of Vermont,Psychology,University of Vermont,64095.375,US,education,2829240,Biology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Illinois Urbana Champaign,337282.56,US,education,17815631,Physics
3879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre Hospitalier Universitaire de Li ge,43428.79,BE,healthcare,311791,Medicine
3880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Xi an Jiaotong University,136077.8,CN,education,4774778,Physics
3881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Royal Children s Hospital,54855.98,AU,healthcare,1489784,Medicine


## Retrieving Journal information

In [157]:
journal_list = combined_data['full'].unique()

In [158]:
combined_data

,cord_uid,display_name,title,license,publish_time,full,relevance_score,works_count,cited_by_count,last_known_institution,research_field
0,ug7v899j,Tariq A Madani,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,3084.6128,107,5240,King Abdulaziz University,Medicine
1,ug7v899j,Aisha A Al-Ghamdi,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,548.21704,29,122,King Abdulaziz University,Medicine
2,02tnwd4m,Albert van der Vliet,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,5049.549,246,10445,University of Vermont,Biology
3,02tnwd4m,Jason P Eiserich,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,4635.8936,133,8770,"University of California, Davis",Biology
4,02tnwd4m,Carroll E Cross,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,4898.441,320,14245,"University of California, Davis",Medicine
...,...,...,...,...,...,...,...,...,...,...,...
3674,NaN,Wayne Jun-Wei Zhou,NaN,NaN,NaN,NaN,161.24713,6,27,Chinese University of Hong Kong,Genetics
3675,NaN,C W Ma Ronald,NaN,NaN,NaN,NaN,69.7828,6,6,Unknown,Genetics
3676,NaN,Tao Qian,NaN,NaN,NaN,NaN,5364.3755,664,6897,Macau University of Science and Technology,Physics
3677,NaN,Yu-Zhen Chen,NaN,NaN,NaN,NaN,2393.0342,218,4903,Jinzhong University,Biology


In [159]:
journal_list

array(['BMC Infect Dis', 'Respiratory Research', 'Virology Journal',
       'The EMBO Journal', 'Critical Care',
       'Journal of the American Medical Informatics Association',
       'BMC Public Health', 'EMBO J', 'Nucleic Acids Research',
       'Biological Procedures Online',
       'Journal of Translational Medicine', 'Retrovirology',
       'BMC Genomics', 'Aust New Zealand Health Policy',
       'Microbial Cell Factories',
       'Journal of Biomedicine and Biotechnology', 'BMC Gastroenterol',
       'PLoS Biol', 'International Journal of Health Geographics',
       'Global Health', 'Biochemistry', 'Immunome Res', 'BMC Mol Biol',
       'EMBO reports', 'PLoS Med', 'BMC Biotechnol',
       'Harm Reduction Journal', 'Nature Medicine', 'PLoS Pathog',
       'BMC Med Ethics', 'Evid Based Complement Alternat Med',
       'PLoS Comput Biol', 'PLoS One', 'Ann Clin Microbiol Antimicrob',
       'Methods', 'Methods-a Companion to Methods in Enzymology',
       'Reprod Biol Endocrinol',


In [160]:
journal_df = pd.DataFrame()

for i in range(len(journal_list)):
    try:
        print(journal_list[i])
        journal_information = requests.get(
            'https://api.openalex.org/venues?filter=display_name.search:'+journal_list[i]
        ).json()['results'][0]
        current_journal = pd.DataFrame.from_dict(journal_information, orient='index')
        current_journal = current_journal.transpose()
        journal_df = journal_df.append(current_journal)
    except:
        print("Empty name")
    #author_data = author_data.append(pd.json_normalize(author_information)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])
    #author_data[i] = author_information['x_concepts'][1]['display_name']
    
    #author_data.append(pd.json_normalize(tariq)[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution.display_name', 'x_concepts']])


BMC Infect Dis
Empty name
Respiratory Research
Virology Journal
The EMBO Journal
Critical Care
Journal of the American Medical Informatics Association
BMC Public Health
EMBO J
Empty name
Nucleic Acids Research
Biological Procedures Online
Journal of Translational Medicine
Retrovirology
BMC Genomics
Aust New Zealand Health Policy
Empty name
Microbial Cell Factories
Journal of Biomedicine and Biotechnology
BMC Gastroenterol
Empty name
PLoS Biol
Empty name
International Journal of Health Geographics
Global Health
Biochemistry
Immunome Res
Empty name
BMC Mol Biol
Empty name
EMBO reports
PLoS Med
Empty name
BMC Biotechnol
Empty name
Harm Reduction Journal
Nature Medicine
PLoS Pathog
Empty name
BMC Med Ethics
Empty name
Evid Based Complement Alternat Med
Empty name
PLoS Comput Biol
Empty name
PLoS One
Ann Clin Microbiol Antimicrob
Empty name
Methods
Methods-a Companion to Methods in Enzymology
Empty name
Reprod Biol Endocrinol
Empty name
Theoretical Biology and Medical Modelling
Clinical Che

In [161]:
journal_df

,id,issn_l,issn,display_name,publisher,relevance_score,works_count,cited_by_count,is_oa,is_in_doaj,...,country_code,societies,alternate_titles,abbreviated_title,type,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/V186153782,1465-9921,"[1465-993X, 1465-9921]",Respiratory Research,Springer Science+Business Media,14000.316,3760,115167,True,True,...,GB,[],[RR],None,journal,"[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 35, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T10:22:36.749311,2016-06-24
0,https://openalex.org/V96615428,0022-538X,"[1098-5514, 1070-6321, 0022-538X]",Journal of Virology,American Society for Microbiology,19123.271,49774,3010517,False,False,...,US,[],[JVI],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 40, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T11:04:47.650784,2016-06-24
0,https://openalex.org/V127916151,0261-4189,"[1460-2075, 0261-4189]",The EMBO Journal,Nature Portfolio,86510.5,20945,2655687,False,False,...,DE,[],[European Molecular Biology Organization journ...,None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 17, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T10:24:22.358572,2016-06-24
0,https://openalex.org/V108732581,0090-3493,"[0090-3493, 1530-0293]",Critical Care Medicine,Lippincott Williams & Wilkins,43645.566,49830,1060769,False,False,...,US,[],[],None,journal,"[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 29, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T09:11:15.293463,2016-06-24
0,https://openalex.org/V129839026,1067-5027,"[1067-5027, 1527-974X]",Journal of the American Medical Informatics As...,Oxford University Press,30069.088,4383,199861,False,False,...,GB,[],[JAMIA],None,journal,"[{'id': 'https://openalex.org/C41008148', 'wik...","[{'year': 2023, 'works_count': 6, 'cited_by_co...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T08:47:30.069368,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,https://openalex.org/V41454044,0923-7534,"[0923-7534, 1569-8041]",Annals of Oncology,Oxford University Press,41002.305,53318,748296,False,False,...,NL,[],[],None,journal,"[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 23, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T07:35:42.568813,2016-06-24
0,https://openalex.org/V125119059,1323-7799,"[1440-1843, 1323-7799]",Respirology,Wiley-Blackwell,14110.693,9713,95127,False,False,...,AU,"[{'url': 'http://www.apsresp.org/', 'organizat...",[],None,journal,"[{'id': 'https://openalex.org/C71924100', 'wik...","[{'year': 2023, 'works_count': 14, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T12:31:04.975024,2016-06-24
0,https://openalex.org/V80886111,0141-9838,"[1365-3024, 0141-9838]",Parasite Immunology,Wiley-Blackwell,18543.771,3425,79477,False,False,...,GB,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 11, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T10:46:04.086023,2016-06-24
0,https://openalex.org/V18188872,1618-2642,"[1618-2642, 1618-2650]",Analytical and Bioanalytical Chemistry,Springer Science+Business Media,52677.992,27348,468852,False,False,...,DE,"[{'url': 'https://www.gdch.de/', 'organization...",[Analytical and bioanalytical chemistry],None,journal,"[{'id': 'https://openalex.org/C185592680', 'wi...","[{'year': 2023, 'works_count': 63, 'cited_by_c...",https://api.openalex.org/works?filter=host_ven...,2023-01-30T11:23:51.630703,2016-06-24


In [162]:
journal_data = journal_df[['issn', 'display_name', 'publisher', 'relevance_score', 'cited_by_count', 'cited_by_count','x_concepts']]
journal_data = journal_data.rename(columns = {'display_name':'journal_display_name', 'publisher': 'journal_publisher', 'relevance_score': 'journal_relevance_score', 'cited_by_count' : 'journal_cited_count', 'x_concepts' : 'main_research_field'})

In [163]:
journal_data

,issn,journal_display_name,journal_publisher,journal_relevance_score,journal_cited_count,journal_cited_count,main_research_field
0,"[1465-993X, 1465-9921]",Respiratory Research,Springer Science+Business Media,14000.316,115167,115167,"[{'id': 'https://openalex.org/C71924100', 'wik..."
0,"[1098-5514, 1070-6321, 0022-538X]",Journal of Virology,American Society for Microbiology,19123.271,3010517,3010517,"[{'id': 'https://openalex.org/C86803240', 'wik..."
0,"[1460-2075, 0261-4189]",The EMBO Journal,Nature Portfolio,86510.5,2655687,2655687,"[{'id': 'https://openalex.org/C86803240', 'wik..."
0,"[0090-3493, 1530-0293]",Critical Care Medicine,Lippincott Williams & Wilkins,43645.566,1060769,1060769,"[{'id': 'https://openalex.org/C71924100', 'wik..."
0,"[1067-5027, 1527-974X]",Journal of the American Medical Informatics As...,Oxford University Press,30069.088,199861,199861,"[{'id': 'https://openalex.org/C41008148', 'wik..."
...,...,...,...,...,...,...,...
0,"[0923-7534, 1569-8041]",Annals of Oncology,Oxford University Press,41002.305,748296,748296,"[{'id': 'https://openalex.org/C71924100', 'wik..."
0,"[1440-1843, 1323-7799]",Respirology,Wiley-Blackwell,14110.693,95127,95127,"[{'id': 'https://openalex.org/C71924100', 'wik..."
0,"[1365-3024, 0141-9838]",Parasite Immunology,Wiley-Blackwell,18543.771,79477,79477,"[{'id': 'https://openalex.org/C86803240', 'wik..."
0,"[1618-2642, 1618-2650]",Analytical and Bioanalytical Chemistry,Springer Science+Business Media,52677.992,468852,468852,"[{'id': 'https://openalex.org/C185592680', 'wi..."


In [164]:
for i in range(len(journal_data)):
    try:
        journal_data.iloc[i, journal_data.columns.get_loc('main_research_field')] = (list(journal_data['main_research_field'])[i][0]['display_name'])    
    except:
        journal_data.iloc[i, journal_data.columns.get_loc('main_research_field')] = "Unknown"

Merging the journal data now to our complete dataframe

In [165]:
# renaming columns and replacing special characters e.g. accent
journal_data.journal_display_name = journal_data.journal_display_name.str.replace('.', '')
#journal_data.journal_display_name = journal_data.journal_display_name.str.replace('[^a-zA-Z]', ' ')
#journal_data.journal_display_name = journal_data.journal_display_name.str.replace('  ',' ')

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_51234/3387540538.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  journal_data.journal_display_name = journal_data.journal_display_name.str.replace('.', '')


In [166]:
list(set(combined_data2['full']) - set(journal_data['journal_display_name']))

[nan,
 'Ann Clin Microbiol Antimicrob',
 'Methods-a Companion to Methods in Enzymology',
 'Cases J',
 'Cytokine & Growth Factor Reviews',
 'Malar J',
 'AIDS Res Ther',
 'Proc Biol Sci',
 'BMC Mol Biol',
 'Clinics (Sao Paulo)',
 'BMC Immunol',
 'J Public Health (Oxf)',
 'Global Health',
 'Open Virol J',
 'BMC Gastroenterol',
 'Source Code Biol Med',
 'BMC Pregnancy Childbirth',
 'BMC Res Notes',
 'BMC Pulm Med',
 'Clin Orthop Relat Res',
 'Curr Chem Genomics',
 'Ann Indian Acad Neurol',
 'Immunome Res',
 'PLoS Med',
 'Environ Health',
 'Scand J Trauma Resusc Emerg Med',
 'Minerva',
 'Arch Immunol Ther Exp (Warsz)',
 'Br J Cancer',
 'Molecular & Cellular Proteomics',
 'Mediators Inflamm',
 'BMC Health Serv Res',
 'PLoS One',
 'Chaos',
 'PLoS Genet',
 'BMC Infect Dis',
 'PLoS Pathog',
 'J Oncol',
 'J Antimicrob Chemother',
 'Reprod Biol Endocrinol',
 'IEEE Trans Inf Technol Biomed',
 'PLoS Negl Trop Dis',
 'EMBO reports',
 'J Glob Infect Dis',
 'Hepat Res Treat',
 'Bull Math Biol',
 'Meth

In [167]:
combined_data2[:100]

,cord_uid,display_name,title,license,publish_time,full,relevance_score,works_count,cited_by_count,last_known_institution,research_field,institution_name,institution_relevance_score,country_code,type,institution_cited_by_count,institution_main_research_field
0,ug7v899j,Tariq A Madani,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,3084.6128,107,5240,King Abdulaziz University,Medicine,King Abdulaziz University,81210.69,SA,education,1896975,Biology
1,ug7v899j,Aisha A Al-Ghamdi,Clinical features of culture-proven Mycoplasma...,no-cc,2001-07-04,BMC Infect Dis,548.21704,29,122,King Abdulaziz University,Medicine,King Abdulaziz University,81210.69,SA,education,1896975,Biology
2,02tnwd4m,Albert van der Vliet,Nitric oxide: a pro-inflammatory mediator in l...,no-cc,2000-08-15,Respiratory Research,5049.549,246,10445,University of Vermont,Biology,University of Vermont,64095.375,US,education,2829240,Biology
3,ip32y0j5,Sean A Diehl,Generation of Human Antigen-Specific Monoclona...,cc-by,2010-10-04,PLoS One,2378.744,76,3518,University of Vermont,Medicine,University of Vermont,64095.375,US,education,2829240,Biology
4,NaN,Y Wai Lam,NaN,NaN,NaN,NaN,250.73355,1,36,University of Vermont,Psychology,University of Vermont,64095.375,US,education,2829240,Biology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,gdsfkw1b,Luciana A Ribeiro,Protein secretion in Lactococcus lactis : an e...,cc-by,2005-01-04,Microbial Cell Factories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,gdsfkw1b,Jane E Gabriel,Protein secretion in Lactococcus lactis : an e...,cc-by,2005-01-04,Microbial Cell Factories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,gdsfkw1b,Valeria D Guimaraes,Protein secretion in Lactococcus lactis : an e...,cc-by,2005-01-04,Microbial Cell Factories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,gdsfkw1b,Maricê N Oliveira,Protein secretion in Lactococcus lactis : an e...,cc-by,2005-01-04,Microbial Cell Factories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
combined_data3 = combined_data2.merge(journal_data, left_on = 'full', right_on = 'journal_display_name', how = 'outer')
combined_data3 = combined_data3.loc[:,~combined_data3.columns.duplicated()]

### Our completed Dataframe consisting of: 



In [169]:
combined_data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3905 entries, 0 to 3904
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   cord_uid                         3210 non-null   object
 1   display_name                     3704 non-null   object
 2   title                            3210 non-null   object
 3   license                          3210 non-null   object
 4   publish_time                     3210 non-null   object
 5   full                             3210 non-null   object
 6   relevance_score                  2959 non-null   object
 7   works_count                      3049 non-null   object
 8   cited_by_count                   3049 non-null   object
 9   last_known_institution           3049 non-null   object
 10  research_field                   3049 non-null   object
 11  institution_name                 2526 non-null   object
 12  institution_relevance_score      2

In [170]:
combined_data3.to_csv('outer_full_dataframe_500_new.csv', index=False, header=True)
